In [2]:
pip install pandas openpyxl lxml



[notice] A new release of pip available: 22.2.2 -> 22.3.1
[notice] To update, run: python3.10 -m pip install --upgrade pip
Note: you may need to restart the kernel to use updated packages.


In [3]:
import pandas as pd
from lxml import etree
from itertools import islice
from collections import defaultdict

In [4]:
PPC_BOM_PATH = 'ppc_bom.xlsx'
PPC_BOM_SHEET = 'ReportUnitCost20221212'
PPC_CLEANED_BOM_PATH = 'ppc_bom_cleaned.csv'

In [5]:
ppc_bom = pd.read_excel(PPC_BOM_PATH, sheet_name=PPC_BOM_SHEET, header = 2, usecols="B:E,H:Z,AK,BI", na_values=['', ' '])
# ppc_bom_small = pd.read_excel(PPC_BOM_PATH, sheet_name=PPC_BOM_SHEET, header = 2, usecols="B:C,H,J,L,M,P,R,T,V:X,AK")

In [6]:
ppc_bom.columns = ppc_bom.columns.str.replace(r'\n', ' ', regex=True)
# ppc_bom.rename(columns={'SYS PARTS/ CAR': 'Quantity', 'PARTNO': 'PartNumber', } )
ppc_bom.head()

,COST CARR,D1,D1 TITLE,Product Stream,SU,SU TITLE,KU,KU TITLE,KU STATUS,RESP DEPT,...,CM NAME,FGRP,FGRP NAME,POS,PARTNO,PARTNO SUFF,PART NAME,DONOR,SYS PARTS/ CAR,Product Module
0,V536X,292250,VEHICLE TOPHAT,VEHICLE TOPHAT,292284,EXTERIOR FRONT AND REAR,307274,Hood and fender systems,DECID,93520,...,P8 US Inscription 7 seats,8210.0,BONNET,50.0,30622598,000,"FLANGE SCREW M6*14*17,2",MBOM,4.0,Exterior Front & Rear
1,V536X,292250,VEHICLE TOPHAT,VEHICLE TOPHAT,292284,EXTERIOR FRONT AND REAR,307274,Hood and fender systems,DECID,93520,...,P8 US Inscription 7 seats,8250.0,FRONT FENDER,10.0,30622598,000,"FLANGE SCREW M6*14*17,2",MBOM,12.0,Exterior Front & Rear
2,MEP2P,292253,ELECTRIC PROPULSION,ELECTRIC PROPULSION,292300,THERMAL MANAGEMENT,302178,Cooling system V536,DECID,97400,...,P8 US Inscription 7 seats,2681.0,NaN,50.0,30622624,000,RETAINER 22,MBOM,1.0,Thermal Exchange & Distribution
3,V536X,292248,SOFTWARE AND ELECTRONICS PLATFORM,SOFTWARE AND ELECTRONICS PLATFORM,292271,CORE SYSTEM COMPONENTS,292340,Core system components,DECID,94200,...,P8 US Inscription 7 seats,3725.0,NaN,50.0,30622876,000,SIX POINT SOCKET SCREW M5,MBOM,3.0,Computing Infrastructure
4,V536X,292251,VEHICLE PLATFORM,VEHICLE PLATFORM,292292,BODY CONTROL AND ROAD CONTACT,280313,Tuning & control,DECID,92520,...,P8 US Inscription 7 seats,7612.0,SHOCK ABSORBER,10.0,30622916,000,SEMS SCREW M8*30*MATHREAD,MBOM,4.0,Body Control


In [7]:
# display(ppc_bom.dtypes)

for index, dtype in enumerate(ppc_bom.dtypes):
    if pd.api.types.is_object_dtype(dtype):
        col_name = ppc_bom.columns[index]
        print(f"changing {col_name}'s type {dtype} to string")
        ppc_bom[col_name] = ppc_bom[col_name].astype("string") 

ppc_bom.dtypes


changing COST CARR's type object to string
changing D1 TITLE's type object to string
changing Product Stream's type object to string
changing SU TITLE's type object to string
changing KU TITLE's type object to string
changing KU STATUS's type object to string
changing PSS NAME's type object to string
changing ART name's type object to string
changing CM NAME's type object to string
changing FGRP NAME's type object to string
changing PARTNO SUFF's type object to string
changing PART NAME's type object to string
changing DONOR's type object to string
changing Product Module's type object to string


COST CARR          string
D1                  int64
D1 TITLE           string
Product Stream     string
SU                  int64
SU TITLE           string
KU                  int64
KU TITLE           string
KU STATUS          string
RESP DEPT           int64
PSSNO               int64
PSS NAME           string
ART no            float64
ART name           string
CMNO                int64
CM NAME            string
FGRP              float64
FGRP NAME          string
POS               float64
PARTNO              int64
PARTNO SUFF        string
PART NAME          string
DONOR              string
SYS PARTS/ CAR    float64
Product Module     string
dtype: object

In [10]:
print(f'Number of rows before clean: {ppc_bom.index.size}')
# Remove Zero Quantity values
ppc_bom_cleaned = pd.DataFrame(ppc_bom[ppc_bom['SYS PARTS/ CAR'] != 0.0])
print(f'Number of rows after clean: {ppc_bom_cleaned.index.size}')


Number of rows before clean: 21969
Number of rows after clean: 7002


In [26]:
ppc_bom_cleaned["FGRP"] = pd.to_numeric(ppc_bom_cleaned["FGRP"],downcast="integer")
ppc_bom_cleaned["POS"] = pd.to_numeric(ppc_bom_cleaned["POS"],downcast="integer")
ppc_bom_cleaned["ART no"] = pd.to_numeric(ppc_bom_cleaned["ART no"],downcast="integer").astype(pd.Int64Dtype())
ppc_bom_cleaned["SYS PARTS/ CAR"] = pd.to_numeric(ppc_bom_cleaned["SYS PARTS/ CAR"]).astype(pd.Float64Dtype())

ppc_bom_cleaned.head()




,COST CARR,D1,D1 TITLE,Product Stream,SU,SU TITLE,KU,KU TITLE,KU STATUS,RESP DEPT,...,CM NAME,FGRP,FGRP NAME,POS,PARTNO,PARTNO SUFF,PART NAME,DONOR,SYS PARTS/ CAR,Product Module
0,V536X,292250,VEHICLE TOPHAT,VEHICLE TOPHAT,292284,EXTERIOR FRONT AND REAR,307274,Hood and fender systems,DECID,93520,...,P8 US Inscription 7 seats,8210,BONNET,50,30622598,000,"FLANGE SCREW M6*14*17,2",MBOM,4.0,Exterior Front & Rear
1,V536X,292250,VEHICLE TOPHAT,VEHICLE TOPHAT,292284,EXTERIOR FRONT AND REAR,307274,Hood and fender systems,DECID,93520,...,P8 US Inscription 7 seats,8250,FRONT FENDER,10,30622598,000,"FLANGE SCREW M6*14*17,2",MBOM,12.0,Exterior Front & Rear
2,MEP2P,292253,ELECTRIC PROPULSION,ELECTRIC PROPULSION,292300,THERMAL MANAGEMENT,302178,Cooling system V536,DECID,97400,...,P8 US Inscription 7 seats,2681,<NA>,50,30622624,000,RETAINER 22,MBOM,1.0,Thermal Exchange & Distribution
3,V536X,292248,SOFTWARE AND ELECTRONICS PLATFORM,SOFTWARE AND ELECTRONICS PLATFORM,292271,CORE SYSTEM COMPONENTS,292340,Core system components,DECID,94200,...,P8 US Inscription 7 seats,3725,<NA>,50,30622876,000,SIX POINT SOCKET SCREW M5,MBOM,3.0,Computing Infrastructure
4,V536X,292251,VEHICLE PLATFORM,VEHICLE PLATFORM,292292,BODY CONTROL AND ROAD CONTACT,280313,Tuning & control,DECID,92520,...,P8 US Inscription 7 seats,7612,SHOCK ABSORBER,10,30622916,000,SEMS SCREW M8*30*MATHREAD,MBOM,4.0,Body Control


In [27]:
ppc_bom_cleaned.to_csv(PPC_CLEANED_BOM_PATH, index=False, sep=';',decimal=',')

### Read and parse the PLM XML

In [10]:
PLMXML_PATH = 'instance_graf.xml'


In [11]:


plm_xml_df = pd.read_xml(PLMXML_PATH, parser='lxml')

In [16]:
plm_xml_df

,application,label,id,type,UserValue,partRef,UserData,Transform,instanceRefs,name,ruleRefs,Representation
0,Teamcenter,RGQpWtm3hhlKhD/InstanceGraph/,None,None,NaN,None,NaN,None,None,None,None,NaN
1,None,None,id4,TC Specific Properties,NaN,None,NaN,None,None,None,None,NaN
2,None,None,id16,None,NaN,#id14,NaN,1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1,None,None,None,NaN
3,None,None,id23,None,NaN,#id21,NaN,1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1,None,None,None,NaN
4,None,None,id30,None,NaN,#id28,NaN,1 0 0 0 0 1 0 0 0 0 1 0 0 0 0 1,None,None,None,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...
79176,None,None,id392103,None,NaN,None,NaN,None,id392112,VCC110-990-770/3-770-EXTERIOR ACCESSORIES (view),None,NaN
79177,None,None,id392110,None,NaN,None,NaN,None,id392119,"VCC110-8910-900/1-ACCESSORIES,SPOILER (view)",None,NaN
79178,None,None,id392117,None,NaN,None,NaN,None,id392126,PH-VCC110-V03335-01/1-ROOF RAILS / RACKS (view),None,NaN
79179,None,None,id392124,None,NaN,None,NaN,None,id392135,DI-VCC110-34042320-00001/1-LOAD_CARRIER_V536 (...,None,NaN


In [21]:
def flatten_xml(node, key_prefix=()):
    """
    Walk an XML node, generating tuples of key parts and values.
    """

    # Copy tag content if any
    text = (node.text or '').strip()
    if text:
        yield key_prefix, text

    # Copy attributes
    for attr, value in node.items():
        yield key_prefix + (attr,), value

    # Recurse into children
    for child in node:
        yield from flatten_xml(child, key_prefix + (child.tag,))


def dictify_key_pairs(pairs, key_sep='-'):
    """
    Dictify key pairs from flatten_xml, taking care of duplicate keys.
    """
    out = {}

    # Group by candidate key.
    key_map = defaultdict(list)
    for key_parts, value in pairs:
        key_map[key_sep.join(key_parts)].append(value)

    # Figure out the final dict with suffixes if required.
    for key, values in key_map.items():
        if len(values) == 1:  # No need to suffix keys.
            out[key] = values[0]
        else:  # More than one value for this key.
            for suffix, value in enumerate(values, 1):
                out[f'{key}{key_sep}{suffix}'] = value

    return out

In [20]:
path = 'digi_150_VCC110_V536_2021-08-18.plmxml'
parser = etree.XMLParser(encoding='utf-8', recover=True)
tree = etree.parse(path, parser)

In [25]:
rows = [dictify_key_pairs(flatten_xml(row)) for row in tree.iter()]

In [42]:
len(rows)

3120978

In [48]:
selected_rows_iter =islice(rows, 1500000, None)
for row in selected_rows_iter:
    print(row)

In [22]:
iterable = tree.iter()
print(f"TYPE {type(iterable)}" )


TYPE <class 'lxml.etree.ElementDepthFirstIterator'>


In [23]:
# iter = tree.getiterator()
# g = (i for i in iter)

# for node in islice(g, 5, None):
#     print(node)